In [1]:
import pandas as pd

passed_data = []
data_procession = []
with open('/home/ouconstand/data/Virus_Host/Virion.csv','r', encoding='ISO-8859-1') as f:
    for line in f.read().splitlines():
        data = line.split('\t')
        if len(data) < 34:
            passed_data.append(data)
        else:
            if data[8] == '' or data[13] == '' or data[3] == '': # delete non_family or non_virustaxid information data
                pass
            else:
                data_procession.append(data)

data = pd.DataFrame(data_procession[1:-1],columns=data_procession[0])

In [2]:
data.shape

(478970, 34)

In [142]:
len(passed_data)

12

In [ ]:
print(passed_data[0])
print(passed_data[3])
print(passed_data[6])
print(passed_data[9])

In [3]:
with open('/home/ouconstand/wang_0207.txt','r') as f:
    vf = [i.strip('\n') for i in f.readlines()]

In [4]:
with open('/home/ouconstand/ICTV_taxid.txt','r') as f:
    ictv = [i.strip('\n') for i in f.readlines()]

In [5]:
calculate_v = data[data['VirusTaxID'].isin(ictv)]
calculate_v = calculate_v[calculate_v['VirusFamily'].isin(vf)]
print(calculate_v.shape)

(346735, 34)


In [10]:
vf.remove('picobirnaviridae')

In [11]:
len(vf)

27

In [12]:
final_data = pd.DataFrame(columns=['Virus_Family', 'Type', 'Value'])

virus_family = vf

n = 0
for one_family in virus_family:
    
    train_data = calculate_v[calculate_v['VirusFamily'] == one_family]
    virusID_hostfamily = train_data[['VirusTaxID','HostFamily']]
    virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True) # drop duplication
    virusID_hostfamily.reset_index(drop=True) # reset index

    dic = {}
    
    homo_total = 0
    non_homo_total = 0

    homo_trans = 0
    homo_non_trans = 0
    
    non_homo_trans = 0
    non_homo_non_trans = 0

    for i,j in zip(virusID_hostfamily['VirusTaxID'], virusID_hostfamily['HostFamily']):
        dic[i] = dic.get(i, []) + [j]

    for value in dic.values():
        if 'hominidae' in value:
            homo_total += 1
            if len(value) > 1:
                homo_trans += 1

            else:
                homo_non_trans += 1
        else:
            non_homo_total += 1
            if len(value) > 1:
                non_homo_trans += 1
            else:
                non_homo_non_trans += 1

        
    if homo_total != 0:
        final_data.loc[(4*n)+0] = [one_family.capitalize(), 'homo_trans', round(homo_trans/homo_total, 5)]
        final_data.loc[(4*n)+1] = [one_family.capitalize(), 'homo_non_trans', round(homo_non_trans/homo_total, 5)]
    if non_homo_total != 0:
        final_data.loc[(4*n)+2] = [one_family.capitalize(), 'non_homo_trans', -round(non_homo_trans/non_homo_total, 5)]
        final_data.loc[(4*n)+3] = [one_family.capitalize(), 'non_homo_non_trans', -round(non_homo_non_trans/non_homo_total, 5)]
    n += 1

<ipython-input-12-4a0bcc0b285d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True) # drop duplication
<ipython-input-12-4a0bcc0b285d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True) # drop duplication
<ipython-input-12-4a0bcc0b285d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [13]:
final_data.to_csv('/mnt/c/Users/ouyangkang/Desktop/final_data.csv',index=None)